<a href="https://colab.research.google.com/github/ewuerfel66/DS-Unit-2-Linear-Models/blob/master/EricWuerfel_DS_Sprint_Challenge_5_Linear_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Linear Models  Sprint Challenge: Predict home prices in Ames, Iowa 🏠💸

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

For your Sprint Challenge, you'll use historical housing data. There's a data dictionary at the bottom of the notebook. 

Run this code cell to load the dataset:





In [0]:
# !pip install category-encoders
# !pip install pandas-profiling

In [0]:
import pandas as pd
import pandas_profiling
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import category_encoders as ce

In [0]:
URL = 'https://drive.google.com/uc?export=download&id=1522WlEW6HFss36roD_Cd9nybqSuiVcCK'
df = pd.read_csv(URL)
assert df.shape == (2904, 47)

In [0]:
# Encode `Central_Air` and `Paved_Drive`
df = df.replace(to_replace='Y', value=1)
df = df.replace(to_replace='N', value=0)
df = df.replace(to_replace='P', value=0.5)

In [0]:
df.head()

,1st_Flr_SF,Bedroom_AbvGr,Bldg_Type,Bsmt_Full_Bath,Bsmt_Half_Bath,Central_Air,Condition_1,Condition_2,Electrical,Exter_Cond,Exter_Qual,Exterior_1st,Exterior_2nd,Foundation,Full_Bath,Functional,Gr_Liv_Area,Half_Bath,Heating,Heating_QC,House_Style,Kitchen_AbvGr,Kitchen_Qual,Land_Contour,Land_Slope,Lot_Area,Lot_Config,Lot_Shape,MS_SubClass,MS_Zoning,Mas_Vnr_Type,Mo_Sold,Neighborhood,Overall_Cond,Overall_Qual,Paved_Drive,Roof_Matl,Roof_Style,SalePrice,Sale_Condition,Sale_Type,Street,TotRms_AbvGrd,Utilities,Year_Built,Year_Remod/Add,Yr_Sold
0,1656,3,1Fam,1.0,0.0,1,Norm,Norm,SBrkr,TA,TA,BrkFace,Plywood,CBlock,1,Typ,1656,0,GasA,Fa,1Story,1,TA,Lvl,Gtl,31770,Corner,IR1,20,RL,Stone,5,NAmes,5,6,0.5,CompShg,Hip,215000,Normal,WD,Pave,7,AllPub,1960,1960,2010
1,896,2,1Fam,0.0,0.0,1,Feedr,Norm,SBrkr,TA,TA,VinylSd,VinylSd,CBlock,1,Typ,896,0,GasA,TA,1Story,1,TA,Lvl,Gtl,11622,Inside,Reg,20,RH,None,6,NAmes,6,5,1.0,CompShg,Gable,105000,Normal,WD,Pave,5,AllPub,1961,1961,2010
2,1329,3,1Fam,0.0,0.0,1,Norm,Norm,SBrkr,TA,TA,Wd Sdng,Wd Sdng,CBlock,1,Typ,1329,1,GasA,TA,1Story,1,Gd,Lvl,Gtl,14267,Corner,IR1,20,RL,BrkFace,6,NAmes,6,6,1.0,CompShg,Hip,172000,Normal,WD,Pave,6,AllPub,1958,1958,2010
3,2110,3,1Fam,1.0,0.0,1,Norm,Norm,SBrkr,TA,Gd,BrkFace,BrkFace,CBlock,2,Typ,2110,1,GasA,Ex,1Story,1,Ex,Lvl,Gtl,11160,Corner,Reg,20,RL,None,4,NAmes,5,7,1.0,CompShg,Hip,244000,Normal,WD,Pave,8,AllPub,1968,1968,2010
4,928,3,1Fam,0.0,0.0,1,Norm,Norm,SBrkr,TA,TA,VinylSd,VinylSd,PConc,2,Typ,1629,1,GasA,Gd,2Story,1,TA,Lvl,Gtl,13830,Inside,IR1,60,RL,None,3,Gilbert,5,5,1.0,CompShg,Gable,189900,Normal,WD,Pave,6,AllPub,1997,1998,2010


## 1. Do train/test split
Train on houses sold in 2006-08. Test on houses sold in 2009-10.

In [0]:
train = df.query('Yr_Sold < 2009')
test = df.query('Yr_Sold >= 2009')

In [0]:
train['Yr_Sold'].value_counts()

2007    686
2006    621
2008    613
Name: Yr_Sold, dtype: int64

In [0]:
test['Yr_Sold'].value_counts()

2009    644
2010    340
Name: Yr_Sold, dtype: int64

In [0]:
# Split into X matrices and y vectors

# Train
X_train = train.drop(columns='SalePrice')
y_train = train[['SalePrice']]

# Test
X_test = test.drop(columns='SalePrice')
y_test = test[['SalePrice']]

## 2. Begin with baselines

What is the Mean Absolute Error and R^2 score for a mean baseline?

Predict `SalePrice`

In [0]:
y_train['SalePrice'].skew()

1.7879659119702855

In [0]:
print(y_train.mean())
print(y_train['SalePrice'].median())

SalePrice    181515.986458
dtype: float64
160500.0


The data is Skewed a bit to the right.

For our baseline model, let's use the median.

In [0]:
y_pred = np.full_like(y_test, fill_value=y_train['SalePrice'].median())

In [0]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2:', r2_score(y_test, y_pred))

MAE: 55282.92581300813
RMSE: 80887.22036542055
R^2: -0.05013866414955359


## 3. Arrange data into X features matrix and y target vector
Select at least one numeric feature and at least one categorical feature.

In [0]:
# Select features

# Find numeric features
X_train_numeric = X_train.select_dtypes('number')
numeric_features = list(X_train_numeric.columns.values)

# Add a categorical feature
features = numeric_features
features.append('Bldg_Type')

In [0]:
# Create X matrix for this model
# The y vector is already created
X_train = X_train[features]
X_test = X_test[features]

In [0]:
X_train.head()

,1st_Flr_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Half_Bath,Central_Air,Full_Bath,Gr_Liv_Area,Half_Bath,Kitchen_AbvGr,Lot_Area,MS_SubClass,Mo_Sold,Overall_Cond,Overall_Qual,Paved_Drive,TotRms_AbvGrd,Year_Built,Year_Remod/Add,Yr_Sold,Bldg_Type
984,1414,3,1.0,0.0,1,1,1414,0,1,11029,20,5,8,6,1.0,6,1958,2002,2008,1Fam
985,2117,4,0.0,0.0,1,2,2117,1,1,12925,20,5,7,6,1.0,7,1970,1970,2008,1Fam
986,1092,4,0.0,0.0,1,2,2112,1,1,11075,60,6,5,6,1.0,9,1969,1969,2008,1Fam
987,926,3,0.0,0.0,1,2,1604,1,1,8702,60,4,5,6,1.0,7,1997,1998,2008,1Fam
988,680,3,0.0,0.0,1,2,1470,1,1,8139,60,10,5,6,1.0,7,1995,1996,2008,1Fam


## 4. Do one-hot encoding

Encode your categorical feature(s).

In [0]:
# OneHotEncode `quantity`
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [0]:
X_test.head()

,1st_Flr_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Half_Bath,Central_Air,Full_Bath,Gr_Liv_Area,Half_Bath,Kitchen_AbvGr,Lot_Area,MS_SubClass,Mo_Sold,Overall_Cond,Overall_Qual,Paved_Drive,TotRms_AbvGrd,Year_Built,Year_Remod/Add,Yr_Sold,Bldg_Type_1Fam,Bldg_Type_TwnhsE,Bldg_Type_Duplex,Bldg_Type_Twnhs,Bldg_Type_2fmCon
0,1656,3,1.0,0.0,1,1,1656,0,1,31770,20,5,5,6,0.5,7,1960,1960,2010,1,0,0,0,0
1,896,2,0.0,0.0,1,1,896,0,1,11622,20,6,6,5,1.0,5,1961,1961,2010,1,0,0,0,0
2,1329,3,0.0,0.0,1,1,1329,1,1,14267,20,6,6,6,1.0,6,1958,1958,2010,1,0,0,0,0
3,2110,3,1.0,0.0,1,2,2110,1,1,11160,20,4,5,7,1.0,8,1968,1968,2010,1,0,0,0,0
4,928,3,0.0,0.0,1,2,1629,1,1,13830,60,3,5,5,1.0,6,1997,1998,2010,1,0,0,0,0


## 5. Use scikit-learn for Linear Regression
Fit your model.

In [0]:
# Instantiate model
model = LinearRegression()

# train model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# Make predictions
y_pred = model.predict(X_test)

## 6. Use regression metrics: MAE, R^2
What is the Mean Absolute Error and R^2 score for your model? Beat the mean baseline!

In [0]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2:', r2_score(y_test, y_pred))

MAE: 21398.96658407548
RMSE: 31846.979592516247
R^2: 0.8372116485928733


## 7. Get coefficients of a linear model
Print or plot the coefficients for the features in your model.

In [0]:
print('Coefficients:', model.coef_)

Coefficients: [[ 1.34144990e+01 -6.30923589e+03  1.81408232e+04  5.54815496e+03
  -4.13621115e+03  3.40917832e+03  6.43942809e+01  1.06158633e+01
  -9.78262499e+03  3.91369040e-01 -2.02242966e+02  2.39275241e+02
   4.72917673e+03  2.20080090e+04 -1.04077502e+03  1.69590099e+01
   4.72283969e+02  1.03880500e+02 -1.55272570e+03 -5.85729343e+02
   1.94850046e+03 -8.21284731e+03 -1.07947936e+04  1.76448698e+04]]


## Stretch Goals
- Try at least 3 different feature combinations.
- Plot exploratory visualizations.
- Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.

### Pandas Profile

In [0]:
pandas_profiling.ProfileReport(df)

Number of variables,47
Number of observations,2904
Total Missing (%),0.0%
Total size in memory,1.0 MiB
Average record size in memory,376.0 B
Numeric,19
Categorical,27
Boolean,1
Date,0
Text (Unique),0
Rejected,0


### Model 1

Add:
* Condition_1
* Electrical
* Foundation
* Heating
* Utilities

In [0]:
# Reset Train and Test sets

# Train
X_train = train.drop(columns='SalePrice')
y_train = train[['SalePrice']]

# Test
X_test = test.drop(columns='SalePrice')
y_test = test[['SalePrice']]

In [0]:
# Select features

# Find numeric features
X_train_numeric = X_train.select_dtypes('number')
numeric_features = list(X_train_numeric.columns.values)

# Add categorical features
features = numeric_features
features.append('Bldg_Type')
features.append('Condition_1')
features.append('Electrical')
features.append('Foundation')
features.append('Heating')
features.append('Utilities')


# Create X matrix for this model
# The y vector is already created
X_train = X_train[features]
X_test = X_test[features]

# OneHotEncode
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# Instantiate model
model = LinearRegression()

# train model
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

In [0]:
print('Training Data:')
print('MAE:', mean_absolute_error(y_train, y_train_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R^2:', r2_score(y_train, y_train_pred))

print(' ')

print('Test Data:')
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2:', r2_score(y_test, y_pred))

Training Data:
MAE: 21924.118372340923
RMSE: 35361.10461003104
R^2: 0.8056013072546634
 
Test Data:
MAE: 21413.964457324895
RMSE: 31624.29378920353
R^2: 0.8394802414895705


### Model 2

Add:
* Exter_Cond
* Exter_Qual
* Mas_Vnr_Type
* Roof_Matl
* Roof_Style

In [0]:
# Reset Train and Test sets

# Train
X_train = train.drop(columns='SalePrice')
y_train = train[['SalePrice']]

# Test
X_test = test.drop(columns='SalePrice')
y_test = test[['SalePrice']]

In [0]:
# Select features

# Find numeric features
X_train_numeric = X_train.select_dtypes('number')
numeric_features = list(X_train_numeric.columns.values)

# Add categorical features
features = numeric_features
features.append('Bldg_Type')
features.append('Condition_1')
features.append('Electrical')
features.append('Foundation')
features.append('Heating')
features.append('Utilities')
features.append('Exter_Cond')
features.append('Exter_Qual')
features.append('Mas_Vnr_Type')
features.append('Roof_Matl')
features.append('Roof_Style')

# Create X matrix for this model
# The y vector is already created
X_train = X_train[features]
X_test = X_test[features]

# OneHotEncode
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# Instantiate model
model = LinearRegression()

# train model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [0]:
print('Training Data:')
print('MAE:', mean_absolute_error(y_train, y_train_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R^2:', r2_score(y_train, y_train_pred))

print(' ')

print('Test Data:')
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2:', r2_score(y_test, y_pred))

Training Data:
MAE: 21924.118372340923
RMSE: 35361.10461003104
R^2: 0.8056013072546634
 
Test Data:
MAE: 19778.095753038175
RMSE: 28192.85901037876
R^2: 0.8724251504654731


### Model 3

Add:
* House_Style
* Neighborhood
* Sale_Condition

In [0]:
# Reset Train and Test sets

# Train
X_train = train.drop(columns='SalePrice')
y_train = train[['SalePrice']]

# Test
X_test = test.drop(columns='SalePrice')
y_test = test[['SalePrice']]

In [0]:
# Select features

# Find numeric features
X_train_numeric = X_train.select_dtypes('number')
numeric_features = list(X_train_numeric.columns.values)

# Add categorical features
features = numeric_features
features.append('House_Style')
features.append('Neighborhood')
features.append('Sale_Condition')
features.append('Bldg_Type')
features.append('Condition_1')
features.append('Electrical')
features.append('Foundation')
features.append('Heating')
features.append('Utilities')
features.append('Exter_Cond')
features.append('Exter_Qual')
features.append('Mas_Vnr_Type')
features.append('Roof_Matl')
features.append('Roof_Style')

# Create X matrix for this model
# The y vector is already created
X_train = X_train[features]
X_test = X_test[features]

# OneHotEncode
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# Instantiate model
model = LinearRegression()

# train model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [0]:
print('Training Data:')
print('MAE:', mean_absolute_error(y_train, y_train_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('R^2:', r2_score(y_train, y_train_pred))

print(' ')

print('Test Data:')
print('MAE:', mean_absolute_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R^2:', r2_score(y_test, y_pred))

Training Data:
MAE: 21924.118372340923
RMSE: 35361.10461003104
R^2: 0.8056013072546634
 
Test Data:
MAE: 18021.905351815014
RMSE: 26511.01856415085
R^2: 0.8871920639291809


## Data Dictionary 

Here's a description of the data fields:

```
1st_Flr_SF: First Floor square feet

Bedroom_AbvGr: Bedrooms above grade (does NOT include basement bedrooms)

Bldg_Type: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
       
Bsmt_Half_Bath: Basement half bathrooms

Bsmt_Full_Bath: Basement full bathrooms

Central_Air: Central air conditioning

       N	No
       Y	Yes
		
Condition_1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition_2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
       
 Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
       
 Exter_Cond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
 
 Exter_Qual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Exterior_1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior_2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
       
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
Full_Bath: Full bathrooms above grade

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Gr_Liv_Area: Above grade (ground) living area square feet
        
Half_Bath: Half baths above grade

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
Heating_QC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

House_Style: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

Kitchen_AbvGr: Kitchens above grade

Kitchen_Qual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Land_Slope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

Lot_Area: Lot size in square feet

Lot_Config: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

Lot_Shape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular

MS_SubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MS_Zoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

Mas_Vnr_Type: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone

Mo_Sold: Month Sold (MM)

Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Overall_Cond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

Overall_Qual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

Paved_Drive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel

Roof_Matl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

Roof_Style: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

SalePrice: the sales price for each house

Sale_Condition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

Sale_Type: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
	
Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
TotRms_AbvGrd: Total rooms above grade (does not include bathrooms)

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
Year_Built: Original construction date

Year_Remod/Add: Remodel date (same as construction date if no remodeling or additions)
						
Yr_Sold: Year Sold (YYYY)	

```